In [3]:
import pandas as pd
import matplotlib.pyplot as plt 
import ast  # To safely evaluate the string representation of tuples
import numpy as np 

from scipy.stats import linregress, t
from scipy import stats
import statsmodels.api as sm
import xlrd
import os
import pymannkendall as mk

import json

In [4]:
csv_file_path = os.path.normpath(os.path.join('.','..','output', 'data','csv','general','number_of_thawed_frozen_trans_day_per_year_month_per_pixel_2000_2021.csv'))


df = pd.read_csv(csv_file_path, header=0, names=['Grid', 'Year', 'Month', 'Status'])

# Convert the JSON-like string to a dictionary and normalize
df_exploded = df['Status'].apply(json.loads)  # Parse JSON strings
df_expanded = pd.json_normalize(df_exploded)  # Expand into separate columns

# Merge the expanded columns back to the original DataFrame
df = pd.concat([df, df_expanded], axis=1)

# Drop the original JSON column if no longer needed
df = df.drop(columns=['Status'])

# Rename the columns
df = df.rename(columns={
    'status_0_count': 'Frozen',
    'status_1_count': 'Thawed'
})



df['Transition'] = df.status_2_count + df.status_3_count
df.drop(columns=['status_2_count', 'status_3_count'], inplace=True)

print(df)
#################### CO
# 0 = AM/PM frozen
# 1 = AM/PM thawed
# 2 = AM frozen, PM thawed (transitional)
# 3 = AM thawed, PM frozen (inverse transitional)

              Grid  Year  Month  Frozen  Thawed  Transition
0        (10, 216)  2000      1      31       0           0
1        (10, 216)  2000      2      29       0           0
2        (10, 216)  2000      3      31       0           0
3        (10, 216)  2000      4      30       0           0
4        (10, 216)  2000      5      29       0           2
...            ...   ...    ...     ...     ...         ...
6552379  (99, 409)  2021      8       0      31           0
6552380  (99, 409)  2021      9       0      30           0
6552381  (99, 409)  2021     10       0      31           0
6552382  (99, 409)  2021     11       0      24           6
6552383  (99, 409)  2021     12       0      25           6

[6552384 rows x 6 columns]


In [7]:
long_lat= pd.read_csv(os.path.join('.', '..', 'output', 'data', 'csv', 'general',  'lat_lon.csv'))
df2 = long_lat.loc[:,['longitude', 'latitude', 'original_indices']]
df2.rename(columns={'original_indices':'Grid'}, inplace=True)

In [9]:
# Remove duplicates and set 'Grid' as the index
df2 = df2.drop_duplicates(subset='Grid').set_index('Grid')

In [12]:
main = pd.merge(df, df2, on='Grid', how='outer')

In [16]:
main = main.dropna()

In [17]:
main.head()

,Grid,Year,Month,Frozen,Thawed,Transition,longitude,latitude
4,"(10, 216)",2000.0,1.0,31.0,0.0,0.0,-123.644,74.2766
5,"(10, 216)",2000.0,2.0,29.0,0.0,0.0,-123.644,74.2766
6,"(10, 216)",2000.0,3.0,31.0,0.0,0.0,-123.644,74.2766
7,"(10, 216)",2000.0,4.0,30.0,0.0,0.0,-123.644,74.2766
8,"(10, 216)",2000.0,5.0,29.0,0.0,2.0,-123.644,74.2766


In [18]:
import pandas as pd
import matplotlib.pyplot as plt

# Ensure Year and Frozen columns are integers
main["Year"] = pd.to_numeric(main["Year"], errors="coerce")
main["Frozen"] = pd.to_numeric(main["Frozen"], errors="coerce")

# Filter for years between 2001 and 2020
filtered_df = main[(main["Year"] >= 2000) & (main["Year"] <= 2020)]


# Group by Grid and Year to calculate total frozen days per year per pixel
frozen_days_per_year = filtered_df.groupby(["Grid", "Year"], as_index=False).agg({"Frozen": "sum"})


# Calculate the average frozen days per year for each pixel
average_frozen_days = frozen_days_per_year.groupby("Grid", as_index=False).agg({"Frozen": "mean"})
print(frozen_days_per_year.shape)


map_data = pd.merge(average_frozen_days, df2, on='Grid', how='outer')


output_path = os.path.join('.', '..', 'output', 'data', 'csv', 'map_frozen_area',  'all_month_frozen_on_average_from_2000_2020_us_included.csv')
map_data.to_csv(output_path)


/var/folders/1p/vx9zfzfn14z2b46713lr78z40000gn/T/ipykernel_94857/1684468578.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main["Year"] = pd.to_numeric(main["Year"], errors="coerce")
/var/folders/1p/vx9zfzfn14z2b46713lr78z40000gn/T/ipykernel_94857/1684468578.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main["Frozen"] = pd.to_numeric(main["Frozen"], errors="coerce")


(520845, 3)
